In [5]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# dir_path = './' #local
dir_path = './drive/MyDrive/BAKA/' #colab

In [6]:
#uncomment in colab
from google.colab import drive
drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', header=None, sep=' ')
df = df.drop(columns=[31])
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,152.028,174.270,114.872,135.012,115.821,169.357,108.251,86.5737,86.4014,60.6628,7.49661,368.805,-11.39230,35.4778,372.833,-21.5318,39.6854,348.592,7.46019,38.71440,326.577,19.15310,37.64440,296.277,5.27758,5.69833,303.563,-4.53744,7.62196,12.33870,35.1608
1,106.969,174.270,138.093,122.560,112.017,151.941,139.409,67.5126,93.4567,141.7770,25.46860,371.098,-13.88750,43.5763,381.204,-39.2998,39.0602,356.814,-7.43494,49.22350,343.591,0.97681,54.97550,295.547,-61.05800,8.98806,317.394,-4.13179,85.07790,5.64757,43.2827
2,115.931,174.270,114.917,134.286,175.119,127.521,177.996,58.0159,69.8662,178.0840,49.53920,383.612,-5.43277,56.8145,405.251,-43.7347,35.5285,371.161,-20.33810,34.98540,355.458,-2.39748,82.31940,336.737,-44.98200,13.94950,337.090,-7.44972,63.64330,21.32290,29.9727
3,129.774,90.000,151.379,164.207,147.675,133.193,153.299,53.3455,64.2682,55.6463,68.55800,397.308,-25.93310,57.0065,390.412,-74.1331,42.2847,387.940,-17.66460,43.71540,368.581,6.15326,45.33460,348.796,1.37925,19.70390,370.834,-9.76106,19.74900,17.69820,71.4496
4,168.765,174.270,156.177,141.628,146.640,128.245,177.981,80.4555,66.8833,71.7721,72.51100,366.007,-19.21010,35.8622,344.209,-96.2058,65.4289,368.487,-5.98384,51.30090,373.351,7.91760,45.88860,362.047,13.63630,27.27890,388.642,-8.96836,15.10440,19.16930,97.3733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,149.508,178.660,144.652,141.589,141.211,140.317,173.599,76.2446,75.0304,72.7944,3.43306,473.408,-46.58480,25.7845,466.134,-126.5530,14.1716,497.456,-28.98440,8.21895,510.987,-30.11830,2.87239,522.783,-37.79160,36.79610,518.710,-48.87630,12.26400,7.59416,117.8630
196,149.347,178.758,145.400,142.273,141.907,139.853,173.076,75.6861,74.6439,72.5005,3.29962,472.204,-45.58130,25.2251,464.618,-126.2930,15.1793,496.000,-28.42360,8.89380,509.273,-29.44780,3.12718,520.981,-36.96540,36.67920,517.278,-48.70080,12.25370,7.67346,118.6160
197,148.335,178.866,146.606,143.288,142.679,139.443,172.818,75.3177,74.4949,72.3500,3.41626,470.354,-44.91860,24.3535,463.122,-126.3610,16.1741,493.829,-27.85840,9.43385,506.701,-28.72460,3.23291,518.414,-36.05660,36.29120,515.412,-48.58660,12.25620,7.78125,119.4990
198,146.453,178.656,149.390,144.159,143.569,137.445,173.159,74.5098,73.8029,71.7551,3.72750,468.922,-42.89510,22.5524,461.831,-125.2760,18.1151,491.755,-27.40510,10.54130,504.270,-28.28420,3.47881,515.864,-35.43990,35.86260,513.897,-48.75530,12.21440,7.67485,119.8710


In [7]:
files = os.listdir(dir_path + 'DataCollection')
files

['4', '2', '6', '1', '3', '5']

In [8]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(columns=[31])
        x_test.append(df.to_numpy())
        y_test.append(int(i)-1);
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(columns=[31])
        x_train.append(df.to_numpy())
        y_train.append(int(i)-1);
    
    print(len(samples), ' ', num_tests)
    
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)

205   41
256   51
151   30
301   60
151   30
211   42


In [9]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)

print(x_train)

x_train.shape:  (1021, 200, 31)
y_train.shiape:  (1021,)
x_test.shape:  (254, 200, 31)
y_test.shape:  (254,)
[[[140.263   155.047   153.056   ...  14.6581   10.9596   14.1146 ]
  [139.707   154.679   150.799   ...  14.6448   11.2233   14.2539 ]
  [140.28    153.484   149.955   ...  14.4216   10.9631   14.1897 ]
  ...
  [146.804   158.72    155.316   ...  21.6282   10.9693   17.1487 ]
  [146.373   159.741   156.251   ...  22.2424   11.1586   17.4078 ]
  [145.241   159.593   156.114   ...  23.4668   11.1882   17.5774 ]]

 [[143.97    157.465   154.8     ...  25.9021   11.1725   17.761  ]
  [143.636   157.388   153.438   ...  26.4988   11.1284   17.8446 ]
  [143.836   155.892   152.227   ...  26.6327   11.2525   17.7005 ]
  ...
  [179.259   168.682   167.151   ...  14.1408   11.9837   13.8259 ]
  [176.149   168.91    173.358   ...  13.8633   12.3902   13.2978 ]
  [172.239   169.42    166.983   ...  14.2964   12.968    13.6028 ]]

 [[167.026   168.398   163.434   ...  13.772    12.8347   1

In [10]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [11]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [12]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (1021, 200, 31)
y_train.shiape:  (1021,)
x_test.shape:  (254, 200, 31)
y_test.shape:  (254,)


In [13]:
model = Sequential()
model.add(LSTM(units=200, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(200,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(6, activation='softmax',dtype='float64'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 200)          185600    
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 200)          800       
_________________________________________________________________
dropout (Dropout)            (None, 200, 200)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 200)          320800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200)          800       
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 200)          3

In [14]:
opt = tf.keras.optimizers.Adam(lr=0.0001)

checkpoint_filepath = 'Checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=800,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=80,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

Epoch 1/800
12/12 [==============================] - 48s 1s/step - loss: 1.3952 - accuracy: 0.4987 - val_loss: 1.7343 - val_accuracy: 0.3786
Epoch 2/800
12/12 [==============================] - 11s 953ms/step - loss: 0.2330 - accuracy: 0.9267 - val_loss: 1.6937 - val_accuracy: 0.3786
Epoch 3/800
12/12 [==============================] - 11s 958ms/step - loss: 0.1376 - accuracy: 0.9457 - val_loss: 1.6599 - val_accuracy: 0.3398
Epoch 4/800
12/12 [==============================] - 11s 957ms/step - loss: 0.1135 - accuracy: 0.9653 - val_loss: 1.6251 - val_accuracy: 0.3981
Epoch 5/800
12/12 [==============================] - 11s 956ms/step - loss: 0.0982 - accuracy: 0.9694 - val_loss: 1.5850 - val_accuracy: 0.4272
Epoch 6/800
12/12 [==============================] - 11s 959ms/step - loss: 0.0784 - accuracy: 0.9734 - val_loss: 1.5426 - val_accuracy: 0.3981
Epoch 7/800
12/12 [==============================] - 12s 962ms/step - loss: 0.0590 - accuracy: 0.9739 - val_loss: 1.4921 - val_accuracy: 0.

KeyboardInterrupt: ignored

In [ ]:
# model.save(dir_path + 'Models/gestures.h5')

In [15]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=80)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_custom', save_format='tf')

Evaluate on test data
4/4 [==============================] - 1s 300ms/step - loss: 0.1531 - accuracy: 0.9567
test loss, test acc: [0.15312451124191284, 0.9566929340362549]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_custom/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_custom/assets


In [ ]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

In [17]:
min_max_scaler.data_min_

array([ 9.30245e+01,  8.99999e+01,  9.00456e+01,  9.00455e+01,
        9.00455e+01,  6.54812e+01,  2.57840e+01,  2.77705e+01,
        2.79286e+01,  2.04904e+01, -4.24721e+02,  6.97324e+01,
       -2.97270e+02, -4.20951e+02,  5.95510e+01, -3.15328e+02,
       -4.48325e+02,  6.33254e+01, -3.17749e+02, -4.46258e+02,
        8.67906e+01, -3.13954e+02, -4.57886e+02,  8.58241e+01,
       -3.02162e+02, -4.42757e+02,  1.03148e+02, -2.82183e+02,
        1.86895e-01,  6.48663e-02,  1.07620e-01])